## Ejercicios

1) Subir la base Titanic a Databricks.   
2) Mostrar las primeras 5 filas en pantalla.   
3) ¿Cuántos registros tenemos en la base? ¿Qué comando utilizamos para obtener dicho valor?     
4) ¿Cuál es el valor medio de del campo Fare para aquellas personas que no fallecieron? ¿Tiene algún tipo de correlación con la clase de la persona?    
5) Renombrar la columna "PassengerId" por "passengerId"  
6) Transformar el campo Embarked. Para los valores que figuran "C" asignar el valor Cherbourg, para los valores "Q", asignar el valor Queenstown y para el valor "S", asignar el valor Southampton
7) Eliminar la variable "Cabin".   
8) Calcular la Mediana de la edad abierto por Puerto de embarcación y por Género.   
9) Crear una variable que indique los deciles del campo "Fare".   


In [0]:
from pyspark.sql import SparkSession #Para poder trabajar con Spark vamos a tener que levantar una sesión
import pyspark.sql.functions as f #Esta línea es muy importante ya que aqui se encuentran todas las funciones que vamos a poder usar
from pyspark.sql.types import IntegerType

In [0]:
#Ejercicio 1
file_location = "/FileStore/tables/titanic_train.txt"
df = spark.read.format("csv") \
  .option("inferSchema", "false") \
  .option("header", "true") \
  .option("sep", ",") \
  .load(file_location)

In [0]:
#Ejercicio 2
display(df.limit(5))

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,null,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,null,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,null,S


In [0]:
#Ejercicio 3
df.count()

Out[4]: 891

In [0]:
#Ejercicio 4
display(df.filter(f.col("Survived") == 1).agg(f.mean(f.col("Fare"))))

avg(Fare)
48.39540760233917


In [0]:
display(df.select(f.corr("PClass", "Fare")))

"corr(PClass, Fare)"
-0.5494996199439076


In [0]:
#Ejercicio 5
df = df.withColumnRenamed("PassengerId", "passengerId")

In [0]:
#Ejercicio 6
df = df.withColumn("Embarked", f.when(f.col("Embarked") == "C", "Cherbourg")\
                    .otherwise(f.col("Embarked")))
df = df.withColumn("Embarked", f.when(f.col("Embarked") == "S", "Southampton")\
                    .otherwise(f.col("Embarked")))
df = df.withColumn("Embarked", f.when(f.col("Embarked") == "Q", "Queenstown")\
                    .otherwise(f.col("Embarked")))

In [0]:
#Ejercicio 7
df = df.drop("Cabin")


In [0]:
#Ejercicio 8
display(df.groupBy(["Embarked", "Sex"]).agg(f.median("Age")))

Embarked,Sex,median(Age)
Queenstown,female,21.5
Queenstown,male,30.0
null,female,50.0
Southampton,female,27.0
Cherbourg,male,30.0
Cherbourg,female,24.0
Southampton,male,28.0


In [0]:
#Ejercicio 9
for column in ["Fare"]:
    df = df.withColumn(column, df[column].cast(IntegerType()))

bucketsNumber = 10
buckets = [x / bucketsNumber for x in range(1, bucketsNumber)]
column_name = "Fare"
quantiles = df.approxQuantile(column_name, buckets, 0.0)
df = df.filter(f.col("Age").isNotNull())
df = df.withColumn("{}_quantile".format(column_name), f.when(f.col(column_name) < quantiles[0], 0)\
                    .otherwise(-1))
for x in range(0, len(quantiles)-1):
    df = df.withColumn("{}_quantile".format(column_name), 
                    f.when(f.col(column_name).between(quantiles[x], quantiles[x+1]),x+1)\
                    .otherwise(f.col("{}_quantile".format(column_name))))

df = df.withColumn("{}_quantile".format(column_name), f.when(f.col(column_name) > quantiles[-1], x+2)\
                   .otherwise(f.col("{}_quantile".format(column_name))))

display(df.groupBy("{}_quantile".format(column_name)).count())

Fare_quantile,count
6,80
3,75
5,67
9,79
4,97
8,85
7,70
2,143
0,18
